In [1]:
from scipy.stats import pointbiserialr
import pandas as pd

def get_top_segment_features(df, target_col='Segment', top_n=20, exclude_cols=None):
    """
    Segment A~E 각 클래스별로, 해당 세그먼트 vs 나머지의 point-biserial correlation을 기반으로
    A/B 관련 상위 변수와 C/D/E 관련 상위 변수를 추출합니다 (중복 제거 후 반환).

    Parameters:
    - df: DataFrame
    - target_col: 세그먼트 컬럼명 (기본 'Segment')
    - top_n: 각 클래스별 상위 상관계수 피처 수
    - exclude_cols: 상관계수 계산에서 제외할 컬럼 리스트

    Returns:
    - top_ab: Segment A/B 관련 주요 컬럼 리스트
    - top_cde: Segment C/D/E 관련 주요 컬럼 리스트
    """

    labels = ['A', 'B', 'C', 'D', 'E']
    corr_dict = {}
    
    # 수치형 컬럼 선택
    features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    if exclude_cols:
        features = [col for col in features if col not in exclude_cols]

    # Segment별 상관계수 계산
    for label in labels:
        binary = (df[target_col] == label).astype(int)
        result = {}
        for feature in features:
            try:
                corr, _ = pointbiserialr(df[feature].fillna(0), binary)
                result[feature] = abs(corr)
            except:
                result[feature] = 0
        corr_dict[label] = pd.Series(result).sort_values(ascending=False)

    # A/B 상위 top_n (합집합, 중복 제거)
    top_a = corr_dict['A'].head(top_n).index.tolist()
    top_b = corr_dict['B'].head(top_n).index.tolist()
    top_ab = list(set(top_a + top_b))

    # C/D/E 상위 top_n (합집합, 중복 제거)
    top_c = corr_dict['C'].head(top_n).index.tolist()
    top_d = corr_dict['D'].head(top_n).index.tolist()
    top_e = corr_dict['E'].head(top_n).index.tolist()
    top_cde = list(set(top_c + top_d + top_e))

    return top_ab, top_cde


In [2]:
import pandas as pd

# 파일 경로
file_path = "../../data/통합_train_데이터.parquet"
df = pd.read_parquet(file_path)

In [3]:
top_ab, top_cde = get_top_segment_features(df, exclude_cols=['Segment_encoded', 'ID'])
print("Segment A/B 관련 변수:", top_ab)
print("Segment C/D/E 관련 변수:", top_cde)

c:\Users\somee\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:5535: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rpb, prob = pearsonr(x, y)
c:\Users\somee\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:5535: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rpb, prob = pearsonr(x, y)
c:\Users\somee\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:5535: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rpb, prob = pearsonr(x, y)
c:\Users\somee\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:5535: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rpb, prob = pearsonr(x, y)
c:\Users\somee\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:5535: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rpb, prob = pearsonr(x, y)


Segment A/B 관련 변수: ['할부금액_3M_R12M', '이용금액_할부_무이자_R12M', '이용건수_할부_무이자_R12M', '정상입금원금_B0M', '이용금액_오프라인_R6M', '정상청구원금_B0M', '이용금액_오프라인_R3M', '_1순위카드이용금액', '평잔_일시불_해외_6M', '승인거절건수_입력오류_R3M', '청구금액_R3M', '이용금액_일시불_R12M', '이용금액_할부_무이자_R3M', '이용금액_할부_무이자_R6M', '정상입금원금_B5M', '이용금액_할부_R12M', '마일_적립포인트_R3M', '정상청구원금_B2M', '포인트_마일리지_환산_B0M', '청구금액_B0', '정상입금원금_B2M', '할부건수_무이자_3M_R12M', '정상청구원금_B5M', '청구금액_R6M', '여유_숙박이용금액', '최대이용금액_일시불_R12M', '_1순위업종_이용금액', '잔액_할부_B0M', '할부금액_무이자_3M_R12M', '잔액_할부_무이자_B0M']
Segment C/D/E 관련 변수: ['이용금액_일시불_R3M', '이용금액_R3M_신용체크', '정상입금원금_B0M', '이용금액_오프라인_R6M', '정상청구원금_B0M', '이용건수_신용_R6M', '이용금액_오프라인_R3M', '이용건수_일시불_R12M', '_1순위카드이용금액', '이용금액_오프라인_B0M', '청구금액_R3M', '이용금액_일시불_R12M', '이용금액_R3M_신용', '정상입금원금_B5M', '이용건수_오프라인_B0M', '정상청구원금_B2M', '이용건수_신판_R12M', '청구금액_B0', '정상입금원금_B2M', '이용금액_일시불_B0M', '정상청구원금_B5M', '청구금액_R6M', '최대이용금액_일시불_R12M', '이용가맹점수', '이용건수_신용_R12M', '이용금액_일시불_R6M']


In [4]:
# 범주형 컬럼 전체 목록 (데이터 타입 기준)
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# top_ab / top_cde 중 범주형만 추출
top_ab_cat = [col for col in top_ab if col in categorical_cols]
top_cde_cat = [col for col in top_cde if col in categorical_cols]

print("📌 Segment A/B 관련 범주형 변수:", top_ab_cat)
print("📌 Segment C/D/E 관련 범주형 변수:", top_cde_cat)

📌 Segment A/B 관련 범주형 변수: []
📌 Segment C/D/E 관련 범주형 변수: []


In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

def get_vif(df, feature_list):
    X = df[feature_list].copy()
    X = X.dropna()  # 결측 제거

    vif_df = pd.DataFrame()
    vif_df["feature"] = X.columns
    vif_df["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_df.sort_values("VIF", ascending=False)

vif_ab = get_vif(df, top_ab)
vif_cde = get_vif(df, top_cde)

print("📌 Segment A/B 관련 변수 VIF")
display(vif_ab)

print("📌 Segment C/D/E 관련 변수 VIF")
display(vif_cde)


📌 Segment A/B 관련 변수 VIF


,feature,VIF
1,이용금액_할부_무이자_R12M,63.671890
15,이용금액_할부_R12M,53.509000
28,할부금액_무이자_3M_R12M,48.445604
17,정상청구원금_B2M,45.759911
5,정상청구원금_B0M,45.604345
2,이용건수_할부_무이자_R12M,44.181988
21,할부건수_무이자_3M_R12M,41.939402
0,할부금액_3M_R12M,39.618459
10,청구금액_R3M,36.726000
22,정상청구원금_B5M,28.828130


📌 Segment C/D/E 관련 변수 VIF


,feature,VIF
16,이용건수_신판_R12M,22153.073986
24,이용건수_신용_R12M,19750.124196
7,이용건수_일시불_R12M,4155.970844
0,이용금액_일시불_R3M,133.738925
19,이용금액_일시불_B0M,98.660557
25,이용금액_일시불_R6M,86.122315
5,이용건수_신용_R6M,65.565066
4,정상청구원금_B0M,50.932335
15,정상청구원금_B2M,47.877691
14,이용건수_오프라인_B0M,40.776563


In [10]:
# 📌 PC1~PC5에서 반복적으로 중요한 변수:
pca_cols = ['CA이자율_할인전', 'CL이자율_할인전', 'RV_평균잔액_R3M', 'RV일시불이자율_할인전', 'RV최소결제비율', 'RV현금서비스이자율_할인전', '방문월수_앱_R6M', '방문일수_앱_B0M', '방문일수_앱_R6M', '방문횟수_앱_B0M', '방문후경과월_앱_R6M', '이용금액_R3M_신용', '이용금액_R3M_신용체크', '이용금액_일시불_B0M', '이용금액대', '일시불ONLY전환가능여부', '잔액_리볼빙일시불이월_B0M', '잔액_일시불_B0M', '잔액_일시불_B1M', '잔액_일시불_B2M', '잔액_카드론_B0M', '잔액_카드론_B1M', '잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M', '잔액_카드론_B5M', '정상청구원금_B0M', '정상청구원금_B2M', '정상청구원금_B5M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '최종카드론_대출금액', '카드론이용금액_누적', '평잔_RV일시불_3M', '평잔_RV일시불_6M', '평잔_일시불_3M', '평잔_일시불_6M', '평잔_카드론_3M', '평잔_카드론_6M', '평잔_할부_3M', '홈페이지_금융건수_R3M', '홈페이지_금융건수_R6M', '홈페이지_선결제건수_R3M', '홈페이지_선결제건수_R6M']

In [17]:
selected_cols=(top_ab + top_cde + pca_cols)
selected_cols = list(dict.fromkeys(selected_cols))

In [27]:
top_cde

['이용금액_일시불_R3M',
 '이용금액_R3M_신용체크',
 '정상입금원금_B0M',
 '이용금액_오프라인_R6M',
 '정상청구원금_B0M',
 '이용건수_신용_R6M',
 '이용금액_오프라인_R3M',
 '이용건수_일시불_R12M',
 '_1순위카드이용금액',
 '이용금액_오프라인_B0M',
 '청구금액_R3M',
 '이용금액_일시불_R12M',
 '이용금액_R3M_신용',
 '정상입금원금_B5M',
 '이용건수_오프라인_B0M',
 '정상청구원금_B2M',
 '이용건수_신판_R12M',
 '청구금액_B0',
 '정상입금원금_B2M',
 '이용금액_일시불_B0M',
 '정상청구원금_B5M',
 '청구금액_R6M',
 '최대이용금액_일시불_R12M',
 '이용가맹점수',
 '이용건수_신용_R12M',
 '이용금액_일시불_R6M']

In [18]:
print(len(selected_cols))         
print(type(selected_cols[0]))

78
<class 'str'>


In [21]:
def map_categorical_columns(df, verbose=True):
    """
    미리 정의된 매핑 기준에 따라 범주형 컬럼들을 수치형으로 변환합니다.
    처리 컬럼: 거주시도명, 연회비발생카드수_B0M, 한도증액횟수_R12M, 이용금액대,
              할인건수_R3M, 할인건수_B0M, 방문횟수_PC_R6M, 방문횟수_앱_R6M, 방문일수_PC_R6M
    """

    # 1. 거주시도명 → 수도권 여부
    capital_area = ['서울특별시', '경기도', '인천광역시']
    if '거주시도명' in df.columns:
        df['거주시도_수도권여부'] = df['거주시도명'].apply(lambda x: 1 if x in capital_area else 0)
        df.drop(columns=['거주시도명'], inplace=True)
        if verbose: print("[거주시도명] → 수도권 여부 인코딩 완료")

    # 2. 연회비발생카드수_B0M
    mapping = {"0개": 0, "1개이상": 1}
    if '연회비발생카드수_B0M' in df.columns:
        df['연회비발생카드수_B0M'] = df['연회비발생카드수_B0M'].map(mapping).astype(int)
        if verbose: print("[연회비발생카드수_B0M] 인코딩 완료")

    # 3. 한도증액횟수_R12M
    mapping = {"0회": 0, "1회이상": 1}
    if '한도증액횟수_R12M' in df.columns:
        df['한도증액횟수_R12M'] = df['한도증액횟수_R12M'].map(mapping).astype(int)
        if verbose: print("[한도증액횟수_R12M] 인코딩 완료")

    # 4. 이용금액대 (중간값 기준: 만원 단위)
    mapping = {
        "09.미사용": 0,
        "05.10만원-": 5,
        "04.10만원+": 20,
        "03.30만원+": 40,
        "02.50만원+": 75,
        "01.100만원+": 150
    }
    if '이용금액대' in df.columns:
        df['이용금액대'] = df['이용금액대'].map(mapping)
        if verbose: print("[이용금액대] 중간값 인코딩 완료")

    # 5. 할인건수 인코딩
    discount_map = {
        "1회 이상": 1,
        "10회 이상": 10,
        "20회 이상": 20,
        "30회 이상": 30,
        "40회 이상": 40
    }
    for col in ['할인건수_R3M', '할인건수_B0M']:
        if col in df.columns:
            df[col] = df[col].map(discount_map).astype(int)
            if verbose: print(f"[{col}] 인코딩 완료")

    # 6. 방문횟수 및 방문일수 인코딩
    visit_map = {
        "1회 이상": 1,
        "10회 이상": 10,
        "20회 이상": 20,
        "30회 이상": 30,
        "40회 이상": 40,
        "50회 이상": 50,
        "60회 이상": 60,
        "70회 이상": 70,
        "80회 이상": 80
    }

    visit_cols = ['방문횟수_PC_R6M', '방문횟수_앱_R6M', '방문일수_PC_R6M']
    for col in visit_cols:
        if col in df.columns:
            df[col] = df[col].map(visit_map).astype(int)
            if verbose: print(f"[{col}] 인코딩 완료")

    return df


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

# 1. 데이터 불러오기
df = pd.read_parquet("../../data/통합_train_데이터.parquet")

# 2. 피처 및 타겟 분리
X = df[selected_cols].copy()
y = df["Segment"]

X = X.loc[:, ~X.columns.duplicated()] #중복제거

# 3. 범주형 인코딩
df = map_categorical_columns(df)
cat_cols = X.select_dtypes(include='object').columns.tolist()
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# 4. 결측치 처리
X = pd.DataFrame(SimpleImputer(strategy='mean').fit_transform(X), columns=X.columns)

# 스케일링 (DataFrame 형태 유지)
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 라벨인코딩
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y)

# 6. train-validation 분할
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

# 7. XGBoost 모델 선언 (GPU 가속)
xgb_model = XGBClassifier(
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)

# 8. 학습
xgb_model.fit(X_train, y_train)

# 9. 예측 및 평가
y_pred = xgb_model.predict(X_val)
print(classification_report(y_val, y_pred))


[거주시도명] → 수도권 여부 인코딩 완료
[연회비발생카드수_B0M] 인코딩 완료
[한도증액횟수_R12M] 인코딩 완료
[이용금액대] 중간값 인코딩 완료
[할인건수_R3M] 인코딩 완료
[할인건수_B0M] 인코딩 완료
[방문횟수_PC_R6M] 인코딩 완료
[방문횟수_앱_R6M] 인코딩 완료
[방문일수_PC_R6M] 인코딩 완료


c:\Users\somee\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:13:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\somee\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:13:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\somee\anaconda3\Lib\site-packages\xgboost\core.py:2676: UserWarning: [14:14:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.s

              precision    recall  f1-score   support

           0       0.84      0.24      0.37       194
           1       1.00      0.28      0.43        29
           2       0.71      0.55      0.62     25518
           3       0.68      0.60      0.64     69848
           4       0.93      0.97      0.95    384411

    accuracy                           0.89    480000
   macro avg       0.83      0.53      0.60    480000
weighted avg       0.88      0.89      0.89    480000

